### COVID-19: Predicting unreported infection rate across U.S. counties
Data source <br>
1) https://data.census.gov/cedsci/ <br>
2) https://coronavirus.jhu.edu/

In [1]:
import pandas as pd
import numpy as np 

# PART 1: Population

In [2]:
data = pd.read_csv('pop.csv')

In [3]:
data

,uid,county,state,population
0,0500000US01001,Autauga County,Alabama,55200
1,0500000US01003,Baldwin County,Alabama,208107
2,0500000US01005,Barbour County,Alabama,25782
3,0500000US01007,Bibb County,Alabama,22527
4,0500000US01009,Blount County,Alabama,57645
...,...,...,...,...
3216,0500000US31113,Logan County,Nebraska,886
3217,0500000US31171,Thomas County,Nebraska,645
3218,0500000US30033,Garfield County,Montana,1141
3219,0500000US38065,Oliver County,North Dakota,1837


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 4 columns):
uid           3221 non-null object
county        3221 non-null object
state         3220 non-null object
population    3221 non-null int64
dtypes: int64(1), object(3)
memory usage: 100.8+ KB


In [11]:
data.loc[3220,'state'] = 'United States'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 4 columns):
uid           3221 non-null object
county        3221 non-null object
state         3221 non-null object
population    3221 non-null int64
dtypes: int64(1), object(3)
memory usage: 100.8+ KB


In [10]:
data.to_csv('us_pop_2018.csv')

# PART 2: Age & Gender

In [343]:
df = pd.read_csv('age_sex.csv')

In [344]:
df.columns

Index(['GEO_ID', 'NAME', 'S0101_C01_001E', 'S0101_C01_001M', 'S0101_C02_001E',
       'S0101_C02_001M', 'S0101_C03_001E', 'S0101_C03_001M', 'S0101_C04_001E',
       'S0101_C04_001M',
       ...
       'S0101_C02_038E', 'S0101_C02_038M', 'S0101_C03_038E', 'S0101_C03_038M',
       'S0101_C04_038E', 'S0101_C04_038M', 'S0101_C05_038E', 'S0101_C05_038M',
       'S0101_C06_038E', 'S0101_C06_038M'],
      dtype='object', length=458)

In [325]:
df.head()

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C02_001E,S0101_C02_001M,S0101_C03_001E,S0101_C03_001M,S0101_C04_001E,S0101_C04_001M,...,S0101_C02_038E,S0101_C02_038M,S0101_C03_038E,S0101_C03_038M,S0101_C04_038E,S0101_C04_038M,S0101_C05_038E,S0101_C05_038M,S0101_C06_038E,S0101_C06_038M
0,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total MOE!!Total population,Estimate!!Percent!!Total population,Margin of Error!!Percent MOE!!Total population,Estimate!!Male!!Total population,Margin of Error!!Male MOE!!Total population,Estimate!!Percent Male!!Total population,Margin of Error!!Percent Male MOE!!Total popul...,...,Estimate!!Percent!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent MOE!!PERCENT ALLOCATE...,Estimate!!Male!!PERCENT ALLOCATED!!Age,Margin of Error!!Male MOE!!PERCENT ALLOCATED!!Age,Estimate!!Percent Male!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Male MOE!!PERCENT ALL...,Estimate!!Female!!PERCENT ALLOCATED!!Age,Margin of Error!!Female MOE!!PERCENT ALLOCATED...,Estimate!!Percent Female!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female MOE!!PERCENT A...
1,0500000US01001,"Autauga County, Alabama",55200,*****,(X),(X),26874,147,(X),(X),...,1.3,(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,0500000US01003,"Baldwin County, Alabama",208107,*****,(X),(X),101188,242,(X),(X),...,1.8,(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,0500000US01005,"Barbour County, Alabama",25782,*****,(X),(X),13697,71,(X),(X),...,0.9,(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,0500000US01007,"Bibb County, Alabama",22527,*****,(X),(X),12152,180,(X),(X),...,8.2,(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [326]:
meta = pd.read_csv('agesex_meta.csv')

In [327]:
meta

,GEO_ID,id,Unnamed: 2
0,NAME,Geographic Area Name,NaN
1,S0101_C01_001E,Estimate!!Total!!Total population,NaN
2,S0101_C01_001M,Margin of Error!!Total MOE!!Total population,NaN
3,S0101_C01_002E,Estimate!!Total!!Total population!!AGE!!Under ...,NaN
4,S0101_C01_002M,Margin of Error!!Total MOE!!Total population!!...,NaN
...,...,...,...
452,S0101_C06_036M,Margin of Error!!Percent Female MOE!!Total pop...,NaN
453,S0101_C06_037E,Estimate!!Percent Female!!PERCENT ALLOCATED!!Sex,NaN
454,S0101_C06_037M,Margin of Error!!Percent Female MOE!!PERCENT A...,NaN
455,S0101_C06_038E,Estimate!!Percent Female!!PERCENT ALLOCATED!!Age,NaN


In [328]:
df.iloc[0].unique()

array(['id', 'Geographic Area Name', 'Estimate!!Total!!Total population',
       'Margin of Error!!Total MOE!!Total population',
       'Estimate!!Percent!!Total population',
       'Margin of Error!!Percent MOE!!Total population',
       'Estimate!!Male!!Total population',
       'Margin of Error!!Male MOE!!Total population',
       'Estimate!!Percent Male!!Total population',
       'Margin of Error!!Percent Male MOE!!Total population',
       'Estimate!!Female!!Total population',
       'Margin of Error!!Female MOE!!Total population',
       'Estimate!!Percent Female!!Total population',
       'Margin of Error!!Percent Female MOE!!Total population',
       'Estimate!!Total!!Total population!!AGE!!Under 5 years',
       'Margin of Error!!Total MOE!!Total population!!AGE!!Under 5 years',
       'Estimate!!Percent!!Total population!!AGE!!Under 5 years',
       'Margin of Error!!Percent MOE!!Total population!!AGE!!Under 5 years',
       'Estimate!!Male!!Total population!!AGE!!Under 5 yea

In [329]:
#https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/

In [345]:
use = ['id','Estimate!!Total!!Total population','Geographic Area Name',
       'Estimate!!Male!!Total population','Estimate!!Female!!Total population',
       'Estimate!!Percent!!Total population!!AGE!!15 to 19 years',
       'Estimate!!Percent!!Total population!!AGE!!20 to 24 years',
      'Estimate!!Percent!!Total population!!AGE!!25 to 29 years','Estimate!!Percent!!Total population!!AGE!!30 to 34 years',
      'Estimate!!Percent!!Total population!!AGE!!35 to 39 years','Estimate!!Percent!!Total population!!AGE!!40 to 44 years',
      'Estimate!!Percent!!Total population!!AGE!!45 to 49 years','Estimate!!Percent!!Total population!!AGE!!50 to 54 years',
     'Estimate!!Percent!!Total population!!AGE!!55 to 59 years', 'Estimate!!Percent!!Total population!!AGE!!60 to 64 years',
      'Estimate!!Percent!!Total population!!AGE!!65 to 69 years','Estimate!!Percent!!Total population!!AGE!!70 to 74 years',
      'Estimate!!Percent!!Total population!!AGE!!75 to 79 years','Estimate!!Percent!!Total population!!AGE!!80 to 84 years',
      'Estimate!!Percent!!Total population!!AGE!!85 years and over']

In [347]:
usecol = meta[meta['id'].isin(use)]['GEO_ID'].to_list()
usecol.append('GEO_ID')

In [348]:
usecol

['NAME',
 'S0101_C01_001E',
 'S0101_C02_005E',
 'S0101_C02_006E',
 'S0101_C02_007E',
 'S0101_C02_008E',
 'S0101_C02_009E',
 'S0101_C02_010E',
 'S0101_C02_011E',
 'S0101_C02_012E',
 'S0101_C02_013E',
 'S0101_C02_014E',
 'S0101_C02_015E',
 'S0101_C02_016E',
 'S0101_C02_017E',
 'S0101_C02_018E',
 'S0101_C02_019E',
 'S0101_C03_001E',
 'S0101_C05_001E',
 'GEO_ID']

In [349]:
data2 = df[usecol]
data2.head()

,NAME,S0101_C01_001E,S0101_C02_005E,S0101_C02_006E,S0101_C02_007E,S0101_C02_008E,S0101_C02_009E,S0101_C02_010E,S0101_C02_011E,S0101_C02_012E,S0101_C02_013E,S0101_C02_014E,S0101_C02_015E,S0101_C02_016E,S0101_C02_017E,S0101_C02_018E,S0101_C02_019E,S0101_C03_001E,S0101_C05_001E,GEO_ID
0,Geographic Area Name,Estimate!!Total!!Total population,Estimate!!Percent!!Total population!!AGE!!15 t...,Estimate!!Percent!!Total population!!AGE!!20 t...,Estimate!!Percent!!Total population!!AGE!!25 t...,Estimate!!Percent!!Total population!!AGE!!30 t...,Estimate!!Percent!!Total population!!AGE!!35 t...,Estimate!!Percent!!Total population!!AGE!!40 t...,Estimate!!Percent!!Total population!!AGE!!45 t...,Estimate!!Percent!!Total population!!AGE!!50 t...,Estimate!!Percent!!Total population!!AGE!!55 t...,Estimate!!Percent!!Total population!!AGE!!60 t...,Estimate!!Percent!!Total population!!AGE!!65 t...,Estimate!!Percent!!Total population!!AGE!!70 t...,Estimate!!Percent!!Total population!!AGE!!75 t...,Estimate!!Percent!!Total population!!AGE!!80 t...,Estimate!!Percent!!Total population!!AGE!!85 y...,Estimate!!Male!!Total population,Estimate!!Female!!Total population,id
1,"Autauga County, Alabama",55200,7.0,6.0,6.6,6.2,7.4,5.9,7.0,7.2,7.5,4.9,4.1,4.4,2.7,1.9,1.5,26874,28326,0500000US01001
2,"Baldwin County, Alabama",208107,6.1,5.2,5.7,5.5,6.0,6.2,6.6,7.0,6.9,7.1,6.3,5.5,3.5,2.3,1.9,101188,106919,0500000US01003
3,"Barbour County, Alabama",25782,5.6,6.4,7.2,7.0,6.4,5.6,6.5,6.9,6.4,6.4,5.9,5.1,3.3,2.1,1.6,13697,12085,0500000US01005
4,"Bibb County, Alabama",22527,6.7,6.6,6.9,6.7,6.5,6.4,8.0,7.2,6.6,5.4,5.7,3.7,2.8,2.2,1.9,12152,10375,0500000US01007


In [350]:
col_ = {'GEO_ID':'geo_id','NAME':'county','S0101_C01_001E':'population','S0101_C02_005E':'age15_19','S0101_C02_006E':'age20_24','S0101_C02_007E':'age25_29',
       'S0101_C02_008E':'age30_34', 'S0101_C02_009E':'35_39','S0101_C02_010E':'40_44','S0101_C02_011E':'45_49',
       'S0101_C02_012E':'50_54','S0101_C02_013E':'age55_59','S0101_C02_014E':'age60_64','S0101_C02_015E':'65_69',
       'S0101_C02_016E':'70_74','S0101_C02_017E':'75_79','S0101_C02_018E':'80_84','S0101_C02_019E':'85plus',
       'S0101_C05_001E':'total_female', 'S0101_C03_001E':'total_male'}

In [351]:
data2 = data2.rename(columns=col_)
data2 = data2[1:]
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 1 to 3221
Data columns (total 20 columns):
county          3221 non-null object
population      3221 non-null object
age15_19        3221 non-null object
age20_24        3221 non-null object
age25_29        3221 non-null object
age30_34        3221 non-null object
35_39           3221 non-null object
40_44           3221 non-null object
45_49           3221 non-null object
50_54           3221 non-null object
age55_59        3221 non-null object
age60_64        3221 non-null object
65_69           3221 non-null object
70_74           3221 non-null object
75_79           3221 non-null object
80_84           3221 non-null object
85plus          3221 non-null object
total_male      3221 non-null object
total_female    3221 non-null object
geo_id          3221 non-null object
dtypes: object(20)
memory usage: 503.4+ KB


In [352]:
datacol = data2.columns.to_list()
datacol = datacol[1:19]
datacol[1:16]

['age15_19',
 'age20_24',
 'age25_29',
 'age30_34',
 '35_39',
 '40_44',
 '45_49',
 '50_54',
 'age55_59',
 'age60_64',
 '65_69',
 '70_74',
 '75_79',
 '80_84',
 '85plus']

In [353]:
data2[datacol] = data2[datacol].astype(float)
data2.describe()

,population,age15_19,age20_24,age25_29,age30_34,35_39,40_44,45_49,50_54,age55_59,age60_64,65_69,70_74,75_79,80_84,85plus,total_male,total_female
count,3.221000e+03,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3.221000e+03,3.221000e+03
mean,2.015501e+05,6.474107,6.264204,5.960975,5.839305,5.845514,5.757467,6.103912,6.828966,7.250078,6.894008,5.959485,4.559640,3.314995,2.285129,2.249829,9.921802e+04,1.023321e+05
std,5.697087e+06,1.420987,2.374332,1.344846,1.111407,1.030050,0.989798,0.883146,0.889795,1.262578,1.365781,1.462709,1.323269,1.052472,0.829263,0.921859,2.804990e+06,2.892100e+06
min,7.500000e+01,0.000000,0.000000,0.400000,1.200000,0.000000,1.200000,0.000000,0.000000,1.700000,1.500000,1.500000,1.000000,0.200000,0.000000,0.000000,3.800000e+01,3.700000e+01
25%,1.121500e+04,5.800000,5.200000,5.100000,5.200000,5.200000,5.200000,5.600000,6.400000,6.600000,6.100000,5.000000,3.700000,2.600000,1.800000,1.600000,5.660000e+03,5.575000e+03
50%,2.599200e+04,6.400000,5.800000,5.800000,5.700000,5.800000,5.800000,6.100000,6.900000,7.200000,6.800000,5.800000,4.400000,3.200000,2.200000,2.100000,1.291800e+04,1.299700e+04
75%,6.661300e+04,7.000000,6.700000,6.600000,6.400000,6.400000,6.300000,6.600000,7.300000,7.900000,7.600000,6.600000,5.200000,3.900000,2.700000,2.700000,3.330100e+04,3.369200e+04
max,3.229030e+08,26.400000,28.700000,14.300000,14.100000,10.400000,12.500000,11.800000,13.000000,26.700000,17.500000,17.200000,16.800000,11.100000,10.200000,8.500000,1.589842e+08,1.639188e+08


In [355]:
data2['percent_female'] = 100*data2['total_female']/data2['population']
data2['percent_male'] = 100*data2['total_male']/data2['population']
one = np.ones(len(data2['percent_female']))*100

data2['ageunder15'] = one -  data2[datacol[1:16]].sum(axis=1)
data2['age35_44']  = data2[datacol[5:7]].sum(axis=1)
data2['age45_54'] = data2[datacol[7:9]].sum(axis=1)
data2['age65_74'] = data2[datacol[11:13]].sum(axis=1)
data2['age_75over'] = data2[datacol[13:16]].sum(axis=1)

In [356]:
data = data2.drop(['35_39','40_44','45_49','50_54','65_69','70_74','75_79','80_84','85plus'],axis=1)

In [359]:
data['percent_totalpop'] = 100*data['population']/322903030.0

In [360]:
data

,county,population,age15_19,age20_24,age25_29,age30_34,age55_59,age60_64,total_male,total_female,geo_id,percent_female,percent_male,ageunder15,age35_44,age45_54,age65_74,age_75over,percent_totalpop
1,"Autauga County, Alabama",55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,51.315217,48.684783,19.7,13.3,14.2,8.5,6.1,0.017095
2,"Baldwin County, Alabama",208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,51.376936,48.623064,18.2,12.2,13.6,11.8,7.7,0.064449
3,"Barbour County, Alabama",25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,46.873788,53.126212,17.6,12.0,13.4,11.0,7.0,0.007984
4,"Bibb County, Alabama",22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,46.055844,53.944156,16.7,12.9,15.2,9.4,6.9,0.006976
5,"Blount County, Alabama",57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,50.673953,49.326047,19.2,12.4,13.8,10.7,7.1,0.017852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,"Vieques Municipio, Puerto Rico",8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,50.609965,49.390035,18.4,12.0,12.3,11.8,8.9,0.002716
3218,"Villalba Municipio, Puerto Rico",22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,51.424347,48.575653,17.8,11.5,13.2,9.8,6.0,0.007121
3219,"Yabucoa Municipio, Puerto Rico",34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,51.562271,48.437729,16.2,12.3,13.6,11.2,7.4,0.010576
3220,"Yauco Municipio, Puerto Rico",36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,52.043141,47.956859,16.0,12.2,13.4,11.7,8.6,0.011285


In [361]:
data.to_csv('pop_age_sex.csv')

In [364]:
datatem = pd.read_csv('pop_age_sex.csv')

In [365]:
datatem

,Unnamed: 0,county,population,age15_19,age20_24,age25_29,age30_34,age55_59,age60_64,total_male,total_female,geo_id,percent_female,percent_male,ageunder15,age35_44,age45_54,age65_74,age_75over,percent_totalpop
0,1,"Autauga County, Alabama",55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,51.315217,48.684783,19.7,13.3,14.2,8.5,6.1,0.017095
1,2,"Baldwin County, Alabama",208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,51.376936,48.623064,18.2,12.2,13.6,11.8,7.7,0.064449
2,3,"Barbour County, Alabama",25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,46.873788,53.126212,17.6,12.0,13.4,11.0,7.0,0.007984
3,4,"Bibb County, Alabama",22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,46.055844,53.944156,16.7,12.9,15.2,9.4,6.9,0.006976
4,5,"Blount County, Alabama",57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,50.673953,49.326047,19.2,12.4,13.8,10.7,7.1,0.017852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,3217,"Vieques Municipio, Puerto Rico",8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,50.609965,49.390035,18.4,12.0,12.3,11.8,8.9,0.002716
3217,3218,"Villalba Municipio, Puerto Rico",22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,51.424347,48.575653,17.8,11.5,13.2,9.8,6.0,0.007121
3218,3219,"Yabucoa Municipio, Puerto Rico",34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,51.562271,48.437729,16.2,12.3,13.6,11.2,7.4,0.010576
3219,3220,"Yauco Municipio, Puerto Rico",36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,52.043141,47.956859,16.0,12.2,13.4,11.7,8.6,0.011285


# PART 3: Income per capita

In [194]:
dt = pd.read_csv('income_percapita_org.csv')

In [198]:
dt = dt[1:]
dt

,GEO_ID,NAME,B19301_001E,B19301_001M
3,0500000US28019,"Choctaw County, Mississippi",20589,1079
4,0500000US28057,"Itawamba County, Mississippi",20629,1120
5,0500000US28015,"Carroll County, Mississippi",22567,2233
6,0500000US28043,"Grenada County, Mississippi",21668,1565
7,0500000US28063,"Jefferson County, Mississippi",13374,2461
...,...,...,...,...
3217,0500000US19021,"Buena Vista County, Iowa",26607,1466
3218,0500000US19077,"Guthrie County, Iowa",28953,1108
3219,0500000US19091,"Humboldt County, Iowa",29882,2355
3220,0500000US19183,"Washington County, Iowa",29857,1424


In [205]:
dt['county'] = dt['NAME'].apply(lambda x: x.split(',')[0])

dt['state'] = dt['NAME'].apply(lambda x: x.split(',')[1] if len(x.split(','))>1 else x)
dt= dt.drop('B19301_001M',axis=1)

In [212]:
col_ = {'GEO_ID':'geo_id','B19301_001E':'income_percapita'}
dt = dt.rename(columns=col_)
dt = dt.drop('NAME',axis=1)

In [216]:
dt['income_percapita']= dt['income_percapita'].astype(float)

In [217]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3219 entries, 3 to 3221
Data columns (total 4 columns):
geo_id              3219 non-null object
income_percapita    3218 non-null float64
county              3219 non-null object
state               3219 non-null object
dtypes: float64(1), object(3)
memory usage: 100.7+ KB


In [219]:
dt.to_csv('income_percapita.csv')

# PART 4: Land area

In [368]:
land = pd.read_csv('land.csv')
land

,Unnamed: 0,County,States,LAND AREA(SQMI)
0,0,Autauga,AL,598.73
1,1,Baldwin,AL,1737.68
2,2,Barbour,AL,891.30
3,3,Bibb,AL,624.10
4,4,Blount,AL,646.77
...,...,...,...,...
3140,3140,Sweetwater,WY,10435.46
3141,3141,Teton,WY,4077.70
3142,3142,Uinta,WY,2084.15
3143,3143,Washakie,WY,2241.18


# PART 5 - Merge all Census data

In [381]:
d1 = pd.read_csv('pop_age_sex.csv',index_col= 0)
d2 = pd.read_csv('income_percapita.csv',index_col= 0)
d3 = pd.read_csv('EMPLOYMENT_STATUS.csv',index_col= 0)
d4 = pd.read_csv('GINI_INDEX.csv',index_col= 0)
d5 = pd.read_csv('commute.csv',index_col= 0)
d6 = pd.read_csv('private_hc.csv',index_col= 0)
d7 = pd.read_csv('public_hc.csv',index_col= 0)
#d8 = pd.read_csv('WORKING_STATUS.csv',index_col= 0)
d9 = pd.read_csv('JHU_byCounty_confirmed_covid_0412.csv',index_col= 0)

In [382]:
d1 = d1.drop('county',axis=1)
d1

,population,age15_19,age20_24,age25_29,age30_34,age55_59,age60_64,total_male,total_female,geo_id,percent_female,percent_male,ageunder15,age35_44,age45_54,age65_74,age_75over,percent_totalpop
1,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,51.315217,48.684783,19.7,13.3,14.2,8.5,6.1,0.017095
2,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,51.376936,48.623064,18.2,12.2,13.6,11.8,7.7,0.064449
3,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,46.873788,53.126212,17.6,12.0,13.4,11.0,7.0,0.007984
4,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,46.055844,53.944156,16.7,12.9,15.2,9.4,6.9,0.006976
5,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,50.673953,49.326047,19.2,12.4,13.8,10.7,7.1,0.017852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,50.609965,49.390035,18.4,12.0,12.3,11.8,8.9,0.002716
3218,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,51.424347,48.575653,17.8,11.5,13.2,9.8,6.0,0.007121
3219,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,51.562271,48.437729,16.2,12.3,13.6,11.2,7.4,0.010576
3220,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,52.043141,47.956859,16.0,12.2,13.4,11.7,8.6,0.011285


In [383]:
d2

,geo_id,income_percapita,county,state
3,0500000US28019,20589.0,Choctaw County,Mississippi
4,0500000US28057,20629.0,Itawamba County,Mississippi
5,0500000US28015,22567.0,Carroll County,Mississippi
6,0500000US28043,21668.0,Grenada County,Mississippi
7,0500000US28063,13374.0,Jefferson County,Mississippi
...,...,...,...,...
3217,0500000US19021,26607.0,Buena Vista County,Iowa
3218,0500000US19077,28953.0,Guthrie County,Iowa
3219,0500000US19091,29882.0,Humboldt County,Iowa
3220,0500000US19183,29857.0,Washington County,Iowa


In [384]:
data = d1.merge(d2[['geo_id','county','state','income_percapita']],how='left',on='geo_id')
data

,population,age15_19,age20_24,age25_29,age30_34,age55_59,age60_64,total_male,total_female,geo_id,...,percent_male,ageunder15,age35_44,age45_54,age65_74,age_75over,percent_totalpop,county,state,income_percapita
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,48.684783,19.7,13.3,14.2,8.5,6.1,0.017095,Autauga County,Alabama,29372.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,48.623064,18.2,12.2,13.6,11.8,7.7,0.064449,Baldwin County,Alabama,31203.0
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,53.126212,17.6,12.0,13.4,11.0,7.0,0.007984,Barbour County,Alabama,18461.0
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,53.944156,16.7,12.9,15.2,9.4,6.9,0.006976,Bibb County,Alabama,20199.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,49.326047,19.2,12.4,13.8,10.7,7.1,0.017852,Blount County,Alabama,22656.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,49.390035,18.4,12.0,12.3,11.8,8.9,0.002716,Vieques Municipio,Puerto Rico,11156.0
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,48.575653,17.8,11.5,13.2,9.8,6.0,0.007121,Villalba Municipio,Puerto Rico,10677.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,48.437729,16.2,12.3,13.6,11.2,7.4,0.010576,Yabucoa Municipio,Puerto Rico,9148.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,47.956859,16.0,12.2,13.4,11.7,8.6,0.011285,Yauco Municipio,Puerto Rico,8339.0


In [385]:
cols = data.columns.to_list()
col = {i:i.upper() for i in cols}
col
data = data.rename(columns=col)
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,PERCENT_MALE,AGEUNDER15,AGE35_44,AGE45_54,AGE65_74,AGE_75OVER,PERCENT_TOTALPOP,COUNTY,STATE,INCOME_PERCAPITA
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,48.684783,19.7,13.3,14.2,8.5,6.1,0.017095,Autauga County,Alabama,29372.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,48.623064,18.2,12.2,13.6,11.8,7.7,0.064449,Baldwin County,Alabama,31203.0
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,53.126212,17.6,12.0,13.4,11.0,7.0,0.007984,Barbour County,Alabama,18461.0
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,53.944156,16.7,12.9,15.2,9.4,6.9,0.006976,Bibb County,Alabama,20199.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,49.326047,19.2,12.4,13.8,10.7,7.1,0.017852,Blount County,Alabama,22656.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,49.390035,18.4,12.0,12.3,11.8,8.9,0.002716,Vieques Municipio,Puerto Rico,11156.0
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,48.575653,17.8,11.5,13.2,9.8,6.0,0.007121,Villalba Municipio,Puerto Rico,10677.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,48.437729,16.2,12.3,13.6,11.2,7.4,0.010576,Yabucoa Municipio,Puerto Rico,9148.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,47.956859,16.0,12.2,13.4,11.7,8.6,0.011285,Yauco Municipio,Puerto Rico,8339.0


In [386]:
d3.head()

,STATE,COUNTY,LABOR_FORCE_PARTICIPATION_RATE,LAB_16-19,LAB_20-24,LAB_25-29,LAB_30-34,LAB_35-44,LAB_45-54,LAB_55-59,...,UNEMP_16-19,UNEMP_20-24,UNEMP_25-29,UNEMP_30-34,UNEMP_35-44,UNEMP_45-54,UNEMP_55-59,UNEMP_60-64,UNEMP_65-74,UNEMP_OVER75
GEO_ID,,,,,,,,,,,,,,,,,,,,,
0500000US01001,Alabama,Autauga County,59.3,31.9,70.8,83.0,81.9,78.9,74.9,66.2,...,14.2,12.7,3.6,3.7,3.6,2.4,1.4,3.2,1.5,0.0
0500000US01003,Alabama,Baldwin County,58.5,39.1,77.1,81.3,78.8,81.3,80.5,69.3,...,13.5,9.7,8.6,2.2,2.4,4.4,1.4,2.0,3.2,2.6
0500000US01005,Alabama,Barbour County,46.0,22.4,57.2,72.0,48.3,54.4,59.7,51.7,...,27.2,11.9,20.3,14.1,9.4,5.4,3.4,1.2,3.0,9.2
0500000US01007,Alabama,Bibb County,47.4,16.9,58.4,56.6,58.1,64.5,61.1,56.2,...,30.0,19.1,8.8,16.8,5.0,3.7,0.6,1.9,4.5,0.0
0500000US01009,Alabama,Blount County,48.6,18.5,58.7,81.2,59.0,71.9,63.9,63.0,...,21.5,8.6,2.9,2.3,4.2,3.1,2.5,3.0,2.4,0.0


In [387]:
d8.head()

,STATE,COUNTY,WORK50-52,WORK1-49,WORK0
GEO_ID,,,,,
0500000US01003,Alabama,Baldwin County,57.4,17.7,24.9
0500000US01005,Alabama,Barbour County,42.4,14.3,43.3
0500000US01007,Alabama,Bibb County,45.9,11.8,42.3
0500000US01009,Alabama,Blount County,50.8,10.3,38.9
0500000US01011,Alabama,Bullock County,46.2,16.9,36.9


In [388]:
d4.head()

,STATE,COUNTY,GINI_INDEX
GEO_ID,,,
0500000US28151,Mississippi,Washington County,0.5328
0500000US28111,Mississippi,Perry County,0.4818
0500000US28019,Mississippi,Choctaw County,0.4432
0500000US28057,Mississippi,Itawamba County,0.4299
0500000US28015,Mississippi,Carroll County,0.4489


In [389]:
def clean_data(d5):
    cols5 = d5.columns.to_list()
    cols5.remove('County')
    cols5.remove('States')
    col5 = {i:i.upper() for i in cols5}
    d5 = d5[cols5]
    d5 = d5.rename(columns=col5)
    d5 = d5.rename(columns={'ID': 'GEO_ID'})
    return d5

In [390]:
d5 = clean_data(d5)
d5

,GEO_ID,WORKERS 16 YEARS AND OVER,"CAR, TRUCK, OR VAN","WORKERS PER CAR, TRUCK, OR VAN",PUBLIC TRANSPORTATION (EXCLUDING TAXICAB),WALKED,BICYCLE,"TAXICAB, MOTORCYCLE, OR OTHER MEANS",WORKED AT HOME,WORKED OUTSIDE COUNTY OF RESIDENCE,...,LESS THAN 10 MINUTES,10 TO 14 MINUTES,15 TO 19 MINUTES,20 TO 24 MINUTES,25 TO 29 MINUTES,30 TO 34 MINUTES,35 TO 44 MINUTES,45 TO 59 MINUTES,60 OR MORE MINUTES,MEAN TRAVEL TIME TO WORK (MINUTES)
0,0500000US01001,24428.0,95.2,1.05,0.1,0.6,0.1,1.3,2.8,64.5,...,12.8,10.5,14.2,13.8,9.5,20.0,7.4,6.0,5.7,25.8
1,0500000US01003,91420.0,91.7,1.05,0.1,0.7,0.1,1.3,6.1,20.6,...,10.6,13.2,14.1,13.4,6.6,17.3,7.0,9.9,7.9,27.4
2,0500000US01005,8538.0,94.2,1.07,0.3,2.1,0.0,2.1,1.3,20.9,...,25.4,16.0,12.6,10.9,4.7,10.8,3.4,8.1,8.0,22.7
3,0500000US01007,7946.0,95.5,1.06,0.5,0.6,0.0,0.9,2.6,64.3,...,10.4,14.1,10.3,8.5,5.0,12.1,14.5,16.2,8.9,29.2
4,0500000US01009,21148.0,96.6,1.06,0.1,0.2,0.0,0.8,2.3,67.0,...,11.9,7.7,6.7,8.7,4.5,12.5,8.6,21.4,18.0,35.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0500000US72145,14194.0,95.6,1.02,0.7,1.4,0.2,0.6,1.5,57.5,...,4.9,10.9,13.8,17.5,8.4,14.1,11.6,6.0,12.7,29.7
3216,0500000US72147,2487.0,91.4,1.08,0.0,4.8,0.0,0.0,3.8,1.6,...,8.7,48.3,32.3,7.1,0.8,1.0,0.0,0.0,1.6,14.3
3217,0500000US72149,6357.0,95.7,1.07,0.1,2.3,0.0,0.0,1.9,53.9,...,16.9,11.6,12.5,6.4,4.4,13.2,10.4,10.3,14.4,28.9
3218,0500000US72151,7513.0,97.0,1.05,0.0,1.7,0.6,0.6,0.1,66.9,...,10.9,8.5,13.1,13.7,6.8,17.4,6.2,9.4,13.9,30.5


In [391]:
data = data.merge(d5,how='left',on='GEO_ID')

In [392]:
d6

,id,County,States,With private health insurance,No private health insurance
0,0500000US01001,Autauga County,Alabama,35411,18851
1,0500000US01003,Baldwin County,Alabama,150836,59388
2,0500000US01005,Barbour County,Alabama,10505,11823
3,0500000US01007,Bibb County,Alabama,12489,7996
4,0500000US01009,Blount County,Alabama,38425,19032
...,...,...,...,...,...
1903,0500000US72143,Vega Alta Municipio,Puerto Rico,8872,28193
1904,0500000US72145,Vega Baja Municipio,Puerto Rico,18446,33772
1905,0500000US72149,Villalba Municipio,Puerto Rico,4763,17652
1906,0500000US72151,Yabucoa Municipio,Puerto Rico,8578,25476


In [393]:
d6 = clean_data(d6)
d6.head()

,GEO_ID,WITH PRIVATE HEALTH INSURANCE,NO PRIVATE HEALTH INSURANCE
0,0500000US01001,35411,18851
1,0500000US01003,150836,59388
2,0500000US01005,10505,11823
3,0500000US01007,12489,7996
4,0500000US01009,38425,19032


In [394]:
data = data.merge(d6,how='left',on='GEO_ID')
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,15 TO 19 MINUTES,20 TO 24 MINUTES,25 TO 29 MINUTES,30 TO 34 MINUTES,35 TO 44 MINUTES,45 TO 59 MINUTES,60 OR MORE MINUTES,MEAN TRAVEL TIME TO WORK (MINUTES),WITH PRIVATE HEALTH INSURANCE,NO PRIVATE HEALTH INSURANCE
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,14.2,13.8,9.5,20.0,7.4,6.0,5.7,25.8,35411.0,18851.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,14.1,13.4,6.6,17.3,7.0,9.9,7.9,27.4,150836.0,59388.0
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,12.6,10.9,4.7,10.8,3.4,8.1,8.0,22.7,10505.0,11823.0
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,10.3,8.5,5.0,12.1,14.5,16.2,8.9,29.2,12489.0,7996.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,6.7,8.7,4.5,12.5,8.6,21.4,18.0,35.1,38425.0,19032.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,32.3,7.1,0.8,1.0,0.0,0.0,1.6,14.3,NaN,NaN
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,12.5,6.4,4.4,13.2,10.4,10.3,14.4,28.9,4763.0,17652.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,13.1,13.7,6.8,17.4,6.2,9.4,13.9,30.5,8578.0,25476.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,14.9,8.6,3.4,22.0,3.6,5.7,4.9,23.2,9637.0,25889.0


In [395]:
d7 = clean_data(d7)
d7

,GEO_ID,WITH PUBLIC COVERAGE,NO PUBLIC COVERAGE
0,0500000US01001,20828,33434
1,0500000US01003,73899,136325
2,0500000US01005,12677,9651
3,0500000US01007,8974,11511
4,0500000US01009,20930,36527
...,...,...,...
1903,0500000US72143,27442,9623
1904,0500000US72145,31718,20500
1905,0500000US72149,16964,5451
1906,0500000US72151,25538,8516


In [396]:
data = data.merge(d7,how='left',on='GEO_ID')
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,25 TO 29 MINUTES,30 TO 34 MINUTES,35 TO 44 MINUTES,45 TO 59 MINUTES,60 OR MORE MINUTES,MEAN TRAVEL TIME TO WORK (MINUTES),WITH PRIVATE HEALTH INSURANCE,NO PRIVATE HEALTH INSURANCE,WITH PUBLIC COVERAGE,NO PUBLIC COVERAGE
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,9.5,20.0,7.4,6.0,5.7,25.8,35411.0,18851.0,20828.0,33434.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,6.6,17.3,7.0,9.9,7.9,27.4,150836.0,59388.0,73899.0,136325.0
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,4.7,10.8,3.4,8.1,8.0,22.7,10505.0,11823.0,12677.0,9651.0
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,5.0,12.1,14.5,16.2,8.9,29.2,12489.0,7996.0,8974.0,11511.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,4.5,12.5,8.6,21.4,18.0,35.1,38425.0,19032.0,20930.0,36527.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,0.8,1.0,0.0,0.0,1.6,14.3,NaN,NaN,NaN,NaN
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,4.4,13.2,10.4,10.3,14.4,28.9,4763.0,17652.0,16964.0,5451.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,6.8,17.4,6.2,9.4,13.9,30.5,8578.0,25476.0,25538.0,8516.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,3.4,22.0,3.6,5.7,4.9,23.2,9637.0,25889.0,25727.0,9799.0


In [397]:
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,25 TO 29 MINUTES,30 TO 34 MINUTES,35 TO 44 MINUTES,45 TO 59 MINUTES,60 OR MORE MINUTES,MEAN TRAVEL TIME TO WORK (MINUTES),WITH PRIVATE HEALTH INSURANCE,NO PRIVATE HEALTH INSURANCE,WITH PUBLIC COVERAGE,NO PUBLIC COVERAGE
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,9.5,20.0,7.4,6.0,5.7,25.8,35411.0,18851.0,20828.0,33434.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,6.6,17.3,7.0,9.9,7.9,27.4,150836.0,59388.0,73899.0,136325.0
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,4.7,10.8,3.4,8.1,8.0,22.7,10505.0,11823.0,12677.0,9651.0
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,5.0,12.1,14.5,16.2,8.9,29.2,12489.0,7996.0,8974.0,11511.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,4.5,12.5,8.6,21.4,18.0,35.1,38425.0,19032.0,20930.0,36527.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,0.8,1.0,0.0,0.0,1.6,14.3,NaN,NaN,NaN,NaN
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,4.4,13.2,10.4,10.3,14.4,28.9,4763.0,17652.0,16964.0,5451.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,6.8,17.4,6.2,9.4,13.9,30.5,8578.0,25476.0,25538.0,8516.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,3.4,22.0,3.6,5.7,4.9,23.2,9637.0,25889.0,25727.0,9799.0


In [398]:
data['GEO_ID']

0       0500000US01001
1       0500000US01003
2       0500000US01005
3       0500000US01007
4       0500000US01009
             ...      
3216    0500000US72147
3217    0500000US72149
3218    0500000US72151
3219    0500000US72153
3220         0100000US
Name: GEO_ID, Length: 3221, dtype: object

In [399]:
data.columns

Index(['POPULATION', 'AGE15_19', 'AGE20_24', 'AGE25_29', 'AGE30_34',
       'AGE55_59', 'AGE60_64', 'TOTAL_MALE', 'TOTAL_FEMALE', 'GEO_ID',
       'PERCENT_FEMALE', 'PERCENT_MALE', 'AGEUNDER15', 'AGE35_44', 'AGE45_54',
       'AGE65_74', 'AGE_75OVER', 'PERCENT_TOTALPOP', 'COUNTY', 'STATE',
       'INCOME_PERCAPITA', 'WORKERS 16 YEARS AND OVER', 'CAR, TRUCK, OR VAN',
       'WORKERS PER CAR, TRUCK, OR VAN',
       'PUBLIC TRANSPORTATION (EXCLUDING TAXICAB)', 'WALKED', 'BICYCLE',
       'TAXICAB, MOTORCYCLE, OR OTHER MEANS', 'WORKED AT HOME',
       'WORKED OUTSIDE COUNTY OF RESIDENCE',
       'WORKED OUTSIDE STATE OF RESIDENCE',
       'WORKERS 16 YEARS AND OVER WHO DID NOT WORK AT HOME',
       'LESS THAN 10 MINUTES', '10 TO 14 MINUTES', '15 TO 19 MINUTES',
       '20 TO 24 MINUTES', '25 TO 29 MINUTES', '30 TO 34 MINUTES',
       '35 TO 44 MINUTES', '45 TO 59 MINUTES', '60 OR MORE MINUTES',
       'MEAN TRAVEL TIME TO WORK (MINUTES)', 'WITH PRIVATE HEALTH INSURANCE',
       'NO PRIVATE

In [400]:
data['COUNTY']

0           Autauga County
1           Baldwin County
2           Barbour County
3              Bibb County
4            Blount County
               ...        
3216     Vieques Municipio
3217    Villalba Municipio
3218     Yabucoa Municipio
3219       Yauco Municipio
3220         United States
Name: COUNTY, Length: 3221, dtype: object

In [401]:
d9 = pd.read_csv('JHU_byCounty_confirmed_covid_0412.csv',index_col= 0)
d9 = d9.reset_index()
d9 = d9[d9['iso3'] == 'USA']
d9_filter = d9[(d9['Admin2'] != 'Unassigned')]
#d9_filter = d9[d9['Admin2'] == None]
d9_filter

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,12,12,12,12,12,12,15,17,19,19
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,28,29,29,38,42,44,56,59,66,71
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1,2,2,2,3,3,4,9,9,10
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,4,4,5,7,8,9,9,11,13,16
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,9,10,10,10,10,10,11,12,12,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197,84080054,US,USA,840,80054.0,Out of WV,West Virginia,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3198,84080055,US,USA,840,80055.0,Out of WI,Wisconsin,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3199,84080056,US,USA,840,80056.0,Out of WY,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3200,84088888,US,USA,840,88888.0,NaN,Diamond Princess,US,0.000000,0.000000,...,49,49,49,49,49,49,49,49,49,49


In [402]:
d11 = pd.read_csv('processed_gm_data.csv')
d11.head()

,id,Geographic Area Name,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree,"$1 to $9,999 or loss","$10,000 to $14,999","$15,000 to $24,999",...,"$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",Median income (dollars),Below 100 percent of the poverty level,100 to 149 percent of the poverty level,At or above 150 percent of the poverty level,Householder lived in owner-occupied housing units,Householder lived in renter-occupied housing units
0,0500000US01001,"Autauga County, Alabama",4204.0,12119.0,10552.0,5903.0,4388.0,6339.0,4126.0,6267.0,...,5708.0,3744.0,1436.0,5779.0,29667,8276.0,4579.0,41219.0,40538.0,13425.0
1,0500000US01003,"Baldwin County, Alabama",14310.0,40579.0,46025.0,30431.0,15644.0,24827.0,15759.0,25871.0,...,21975.0,14480.0,5535.0,20333.0,28632,21239.0,16032.0,165641.0,152375.0,50637.0
2,0500000US01005,"Barbour County, Alabama",4901.0,6486.0,4566.0,1417.0,803.0,4808.0,2734.0,3283.0,...,1809.0,1089.0,343.0,1155.0,18138,6332.0,2690.0,13521.0,13662.0,8907.0
3,0500000US01007,"Bibb County, Alabama",2650.0,7471.0,3846.0,1197.0,616.0,2983.0,1886.0,3235.0,...,1605.0,1047.0,704.0,1046.0,22298,2782.0,2347.0,14924.0,15370.0,4651.0
4,0500000US01009,"Blount County, Alabama",7861.0,13489.0,13267.0,3217.0,1793.0,6275.0,4113.0,6795.0,...,5206.0,3861.0,1555.0,3082.0,26509,8084.0,6249.0,42291.0,45751.0,10850.0


In [403]:
def clean_data(d5):
    cols5 = d5.columns.to_list()
    #cols5.remove('County')
    #cols5.remove('States')
    col5 = {i:i.upper() for i in cols5}
    d5 = d5[cols5]
    d5 = d5.rename(columns=col5)
    d5 = d5.rename(columns={'ID': 'GEO_ID'})
    return d5
d11 = clean_data(d11)
d11.columns

Index(['GEO_ID', 'GEOGRAPHIC AREA NAME', 'LESS THAN HIGH SCHOOL GRADUATE',
       'HIGH SCHOOL GRADUATE (INCLUDES EQUIVALENCY)',
       'SOME COLLEGE OR ASSOCIATE'S DEGREE', 'BACHELOR'S DEGREE',
       'GRADUATE OR PROFESSIONAL DEGREE', '$1 TO $9,999 OR LOSS',
       '$10,000 TO $14,999', '$15,000 TO $24,999', '$25,000 TO $34,999',
       '$35,000 TO $49,999', '$50,000 TO $64,999', '$65,000 TO $74,999',
       '$75,000 OR MORE', 'MEDIAN INCOME (DOLLARS)',
       'BELOW 100 PERCENT OF THE POVERTY LEVEL',
       '100 TO 149 PERCENT OF THE POVERTY LEVEL',
       'AT OR ABOVE 150 PERCENT OF THE POVERTY LEVEL',
       'HOUSEHOLDER LIVED IN OWNER-OCCUPIED HOUSING UNITS',
       'HOUSEHOLDER LIVED IN RENTER-OCCUPIED HOUSING UNITS'],
      dtype='object')

In [404]:
#add percent_male, drop total_female, add area!!!, bin age again
#take a look: 'WORKERS 16 YEARS AND OVER' when we have unemployment rate
#impute missing: 
data.columns

Index(['POPULATION', 'AGE15_19', 'AGE20_24', 'AGE25_29', 'AGE30_34',
       'AGE55_59', 'AGE60_64', 'TOTAL_MALE', 'TOTAL_FEMALE', 'GEO_ID',
       'PERCENT_FEMALE', 'PERCENT_MALE', 'AGEUNDER15', 'AGE35_44', 'AGE45_54',
       'AGE65_74', 'AGE_75OVER', 'PERCENT_TOTALPOP', 'COUNTY', 'STATE',
       'INCOME_PERCAPITA', 'WORKERS 16 YEARS AND OVER', 'CAR, TRUCK, OR VAN',
       'WORKERS PER CAR, TRUCK, OR VAN',
       'PUBLIC TRANSPORTATION (EXCLUDING TAXICAB)', 'WALKED', 'BICYCLE',
       'TAXICAB, MOTORCYCLE, OR OTHER MEANS', 'WORKED AT HOME',
       'WORKED OUTSIDE COUNTY OF RESIDENCE',
       'WORKED OUTSIDE STATE OF RESIDENCE',
       'WORKERS 16 YEARS AND OVER WHO DID NOT WORK AT HOME',
       'LESS THAN 10 MINUTES', '10 TO 14 MINUTES', '15 TO 19 MINUTES',
       '20 TO 24 MINUTES', '25 TO 29 MINUTES', '30 TO 34 MINUTES',
       '35 TO 44 MINUTES', '45 TO 59 MINUTES', '60 OR MORE MINUTES',
       'MEAN TRAVEL TIME TO WORK (MINUTES)', 'WITH PRIVATE HEALTH INSURANCE',
       'NO PRIVATE

In [405]:
data = data.merge(d11,how='left',on='GEO_ID')


In [406]:
data[data['WITH PRIVATE HEALTH INSURANCE'].isna() == True]['STATE'].unique()

array([' Alabama', ' Alaska', ' Arizona', ' Arkansas', ' California',
       ' Colorado', ' Florida', ' Georgia', ' Hawaii', ' Idaho',
       ' Illinois', ' Indiana', ' Iowa', ' Kansas', ' Kentucky',
       ' Louisiana', ' Maine', ' Maryland', ' Massachusetts', ' Michigan',
       ' Minnesota', ' Mississippi', nan, ' Missouri', ' Montana',
       ' Nebraska', ' Nevada', ' New Mexico', ' New York',
       ' North Carolina', ' North Dakota', ' Ohio', ' Oklahoma',
       ' Oregon', ' Pennsylvania', ' South Carolina', ' South Dakota',
       ' Tennessee', ' Texas', ' Utah', ' Vermont', ' Virginia',
       ' Washington', ' West Virginia', ' Wisconsin', ' Wyoming',
       ' Puerto Rico', 'United States'], dtype=object)

In [416]:
#d3 = d3.drop('STATE',axis=1)
#d3 = d3.drop('COUNTY',axis=1)

#d3 = d3.reset_index()
#d3 = d3.drop('level_0',axis=1)
#d3 = d3.drop('index',axis=1)
d3

,GEO_ID,LABOR_FORCE_PARTICIPATION_RATE,LAB_16-19,LAB_20-24,LAB_25-29,LAB_30-34,LAB_35-44,LAB_45-54,LAB_55-59,LAB_60-64,...,UNEMP_16-19,UNEMP_20-24,UNEMP_25-29,UNEMP_30-34,UNEMP_35-44,UNEMP_45-54,UNEMP_55-59,UNEMP_60-64,UNEMP_65-74,UNEMP_OVER75
0,0500000US01001,59.3,31.9,70.8,83.0,81.9,78.9,74.9,66.2,52.1,...,14.2,12.7,3.6,3.7,3.6,2.4,1.4,3.2,1.5,0.0
1,0500000US01003,58.5,39.1,77.1,81.3,78.8,81.3,80.5,69.3,48.8,...,13.5,9.7,8.6,2.2,2.4,4.4,1.4,2.0,3.2,2.6
2,0500000US01005,46.0,22.4,57.2,72.0,48.3,54.4,59.7,51.7,44.6,...,27.2,11.9,20.3,14.1,9.4,5.4,3.4,1.2,3.0,9.2
3,0500000US01007,47.4,16.9,58.4,56.6,58.1,64.5,61.1,56.2,52.1,...,30.0,19.1,8.8,16.8,5.0,3.7,0.6,1.9,4.5,0.0
4,0500000US01009,48.6,18.5,58.7,81.2,59.0,71.9,63.9,63.0,39.3,...,21.5,8.6,2.9,2.3,4.2,3.1,2.5,3.0,2.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,0500000US72147,41.5,21.9,74.5,65.2,75.0,58.7,48.5,66.1,11.9,...,0.0,11.6,21.1,11.1,25.9,5.9,14.6,26.9,0.0,-
3217,0500000US72149,46.1,17.8,61.4,83.3,78.4,74.1,58.4,40.1,25.7,...,64.7,56.3,33.1,20.8,11.2,11.4,13.9,4.4,0.0,0.0
3218,0500000US72151,38.7,14.4,62.1,78.8,66.1,65.5,50.4,33.7,11.5,...,70.1,40.0,19.7,25.1,31.1,15.9,15.1,24.4,0.0,0.0
3219,0500000US72153,36.9,6.5,34.8,65.8,58.7,70.1,55.6,31.4,22.5,...,56.3,44.3,19.4,24.3,16.7,17.6,20.4,11.7,0.0,0.0


In [417]:
data = data.merge(d3,how='left',on='GEO_ID')
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,UNEMP_16-19,UNEMP_20-24,UNEMP_25-29,UNEMP_30-34,UNEMP_35-44,UNEMP_45-54,UNEMP_55-59,UNEMP_60-64,UNEMP_65-74,UNEMP_OVER75
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,14.2,12.7,3.6,3.7,3.6,2.4,1.4,3.2,1.5,0.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,13.5,9.7,8.6,2.2,2.4,4.4,1.4,2.0,3.2,2.6
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,27.2,11.9,20.3,14.1,9.4,5.4,3.4,1.2,3.0,9.2
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,30.0,19.1,8.8,16.8,5.0,3.7,0.6,1.9,4.5,0.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,21.5,8.6,2.9,2.3,4.2,3.1,2.5,3.0,2.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,0.0,11.6,21.1,11.1,25.9,5.9,14.6,26.9,0.0,-
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,64.7,56.3,33.1,20.8,11.2,11.4,13.9,4.4,0.0,0.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,70.1,40.0,19.7,25.1,31.1,15.9,15.1,24.4,0.0,0.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,56.3,44.3,19.4,24.3,16.7,17.6,20.4,11.7,0.0,0.0


In [424]:
data['COUNTY'].tail(100)

3121       Campbell County
3122         Carbon County
3123       Converse County
3124          Crook County
3125        Fremont County
               ...        
3216     Vieques Municipio
3217    Villalba Municipio
3218     Yabucoa Municipio
3219       Yauco Municipio
3220         United States
Name: COUNTY, Length: 100, dtype: object

In [427]:
colwrong = ['WORKERS PER CAR, TRUCK, OR VAN','MEAN TRAVEL TIME TO WORK (MINUTES)','MEDIAN INCOME (DOLLARS)',
           'LAB_16-19','LAB_20-24','EMP_45-54','UNEMP_16-19','UNEMP_20-24','UNEMP_25-29','UNEMP_45-54','UNEMP_65-74',
            'UNEMP_OVER75']
#data[colwrong] = data[colwrong].astype(float) -> null value - cannot do

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3221 entries, 0 to 3220
Data columns (total 99 columns):
POPULATION                                            3221 non-null float64
AGE15_19                                              3221 non-null float64
AGE20_24                                              3221 non-null float64
AGE25_29                                              3221 non-null float64
AGE30_34                                              3221 non-null float64
AGE55_59                                              3221 non-null float64
AGE60_64                                              3221 non-null float64
TOTAL_MALE                                            3221 non-null float64
TOTAL_FEMALE                                          3221 non-null float64
GEO_ID                                                3221 non-null object
PERCENT_FEMALE                                        3221 non-null float64
PERCENT_MALE                                          3221

In [430]:
data

,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,AGE60_64,TOTAL_MALE,TOTAL_FEMALE,GEO_ID,...,UNEMP_16-19,UNEMP_20-24,UNEMP_25-29,UNEMP_30-34,UNEMP_35-44,UNEMP_45-54,UNEMP_55-59,UNEMP_60-64,UNEMP_65-74,UNEMP_OVER75
0,55200.0,7.0,6.0,6.6,6.2,7.5,4.9,26874.0,28326.0,0500000US01001,...,14.2,12.7,3.6,3.7,3.6,2.4,1.4,3.2,1.5,0.0
1,208107.0,6.1,5.2,5.7,5.5,6.9,7.1,101188.0,106919.0,0500000US01003,...,13.5,9.7,8.6,2.2,2.4,4.4,1.4,2.0,3.2,2.6
2,25782.0,5.6,6.4,7.2,7.0,6.4,6.4,13697.0,12085.0,0500000US01005,...,27.2,11.9,20.3,14.1,9.4,5.4,3.4,1.2,3.0,9.2
3,22527.0,6.7,6.6,6.9,6.7,6.6,5.4,12152.0,10375.0,0500000US01007,...,30.0,19.1,8.8,16.8,5.0,3.7,0.6,1.9,4.5,0.0
4,57645.0,6.4,5.5,5.9,5.9,6.8,6.3,28434.0,29211.0,0500000US01009,...,21.5,8.6,2.9,2.3,4.2,3.1,2.5,3.0,2.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,8771.0,3.9,6.2,6.0,5.7,7.3,7.5,4332.0,4439.0,0500000US72147,...,0.0,11.6,21.1,11.1,25.9,5.9,14.6,26.9,0.0,-
3217,22993.0,8.1,7.7,6.6,5.5,8.0,5.8,11169.0,11824.0,0500000US72149,...,64.7,56.3,33.1,20.8,11.2,11.4,13.9,4.4,0.0,0.0
3218,34149.0,6.8,6.9,6.2,5.3,6.6,7.5,16541.0,17608.0,0500000US72151,...,70.1,40.0,19.7,25.1,31.1,15.9,15.1,24.4,0.0,0.0
3219,36439.0,6.5,6.3,5.9,5.4,7.8,6.2,17475.0,18964.0,0500000US72153,...,56.3,44.3,19.4,24.3,16.7,17.6,20.4,11.7,0.0,0.0


In [429]:
data.to_csv('final_data_county.csv')